# Validação: BioBERTpt + Focal Loss

**BioBERTpt - Modelo biomédico PT-BR**

## 📊 Histórico
- Score 0.72480 com CrossEntropy (2º melhor transformer)
- **NUNCA testado com Focal Loss** (potencial maior)

## 🎯 Objetivo
Testar BioBERTpt com Focal Loss e outras otimizações.

## 📊 Configurações Testadas
1. **Loss:** CrossEntropy vs Focal Loss (γ=1,2,3)
2. **LR:** 1e-5, 2e-5, 3e-5
3. **Threshold Tuning:** Por classe

---

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from transformers import (
    AutoModelForSequenceClassification, AutoTokenizer,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Detectar ambiente
if os.path.exists('/kaggle/input'):
    DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'
    def find_model_path():
        base = '/kaggle/input'
        def search_dir(directory, depth=0, max_depth=10):
            if depth > max_depth: return None
            try:
                for item in os.listdir(directory):
                    path = os.path.join(directory, item)
                    if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                        return path
                    result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                    if result: return result
            except: pass
            return None
        return search_dir(base)
    MODEL_PATH = find_model_path()
else:
    DATA_DIR = '../data'
    MODEL_PATH = 'pucpr/biobertpt-all'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print(f'Model: {MODEL_PATH}')

In [ ]:
# ===== CARREGAR DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

print(f'Total: {len(train_df)}')
print(f'\nDistribuição targets:')
print(train_df['target'].value_counts().sort_index())

# Split 80/20 estratificado
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].tolist(),
    train_df['target'].tolist(),
    test_size=0.2,
    stratify=train_df['target'],
    random_state=SEED
)

print(f'\nTrain: {len(train_texts)}, Val: {len(val_texts)}')

In [ ]:
# ===== DATASET =====
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )
        item = {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# ===== FOCAL LOSS =====
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, num_classes=7):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

class FocalLossTrainer(Trainer):
    def __init__(self, focal_gamma=2.0, focal_alpha=0.25, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = FocalLoss(gamma=focal_gamma, alpha=focal_alpha)
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop('labels')
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
# ===== FUNÇÃO DE TREINO =====
def train_and_evaluate(config, tokenizer):
    print(f"\n{'='*60}")
    print(f"Config: {config['name']}")
    print(f"{'='*60}")
    
    train_ds = TextDataset(train_texts, train_labels, tokenizer, config['max_length'])
    val_ds = TextDataset(val_texts, val_labels, tokenizer, config['max_length'])
    
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_PATH, num_labels=7, local_files_only=True
    )
    
    def compute_metrics(eval_pred):
        preds = np.argmax(eval_pred.predictions, axis=1)
        f1 = f1_score(eval_pred.label_ids, preds, average='macro')
        return {'f1_macro': f1}
    
    args = TrainingArguments(
        output_dir='/tmp/biobertpt_val',
        num_train_epochs=config.get('epochs', 5),
        per_device_train_batch_size=config['batch_size'],
        per_device_eval_batch_size=config['batch_size']*2,
        learning_rate=config['lr'],
        weight_decay=0.01,
        warmup_ratio=0.1,
        fp16=True,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        report_to='none',
        seed=SEED,
    )
    
    if config.get('focal_gamma'):
        trainer = FocalLossTrainer(
            focal_gamma=config['focal_gamma'],
            focal_alpha=config.get('focal_alpha', 0.25),
            model=model, args=args,
            train_dataset=train_ds, eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )
    else:
        trainer = Trainer(
            model=model, args=args,
            train_dataset=train_ds, eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )
    
    import time
    start = time.time()
    trainer.train()
    train_time = time.time() - start
    
    results = trainer.evaluate()
    results['train_time'] = train_time
    
    preds = trainer.predict(val_ds)
    y_pred = np.argmax(preds.predictions, axis=1)
    
    print(f"F1-Macro: {results['eval_f1_macro']:.5f}")
    print(f"Tempo: {train_time/60:.1f} min")
    print(classification_report(val_labels, y_pred))
    
    # Retornar também logits para threshold tuning
    logits = preds.predictions
    
    del model, trainer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return results, y_pred, logits

In [ ]:
# ===== THRESHOLD TUNING =====
def find_best_thresholds(logits, labels, n_classes=7):
    """Encontra os melhores thresholds por classe."""
    from scipy.special import softmax
    
    probs = softmax(logits, axis=1)
    best_thresholds = {}
    
    for c in range(n_classes):
        best_f1 = 0
        best_t = 0.5
        
        for t in np.arange(0.1, 0.9, 0.05):
            # Aplicar threshold apenas para esta classe
            preds = np.argmax(probs, axis=1)
            
            # Se prob da classe c > t, prediz c
            mask = probs[:, c] > t
            preds_adjusted = preds.copy()
            preds_adjusted[mask] = c
            
            f1 = f1_score(labels, preds_adjusted, average='macro')
            if f1 > best_f1:
                best_f1 = f1
                best_t = t
        
        best_thresholds[c] = best_t
    
    return best_thresholds

In [ ]:
# ===== CARREGAR TOKENIZER =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
print(f'Tokenizer carregado!')

In [ ]:
# ===== EXPERIMENTO 1: CrossEntropy (baseline) =====
config_ce = {'name': 'CrossEntropy', 'lr': 2e-5, 'batch_size': 8, 'max_length': 256}
results_ce, preds_ce, logits_ce = train_and_evaluate(config_ce, tokenizer)

In [ ]:
# ===== EXPERIMENTO 2: Focal Loss γ=2 =====
config_focal = {'name': 'Focal_g2', 'lr': 2e-5, 'batch_size': 8, 'max_length': 256, 'focal_gamma': 2.0}
results_focal, preds_focal, logits_focal = train_and_evaluate(config_focal, tokenizer)

In [ ]:
# ===== EXPERIMENTO 3: Focal Loss γ=3 =====
config_focal3 = {'name': 'Focal_g3', 'lr': 2e-5, 'batch_size': 8, 'max_length': 256, 'focal_gamma': 3.0}
results_focal3, preds_focal3, logits_focal3 = train_and_evaluate(config_focal3, tokenizer)

In [ ]:
# ===== EXPERIMENTO 4: LR mais baixo + Focal =====
config_lr = {'name': 'Focal_lr1e5', 'lr': 1e-5, 'batch_size': 8, 'max_length': 256, 'focal_gamma': 2.0}
results_lr, preds_lr, logits_lr = train_and_evaluate(config_lr, tokenizer)

In [ ]:
# ===== THRESHOLD TUNING no melhor modelo =====
# Usar logits do Focal Loss (ajustar se outro for melhor)
best_logits = logits_focal

thresholds = find_best_thresholds(best_logits, val_labels)
print("\n📊 Thresholds Otimizados:")
for c, t in thresholds.items():
    print(f"  Classe {c}: {t:.2f}")

# Aplicar thresholds
from scipy.special import softmax
probs = softmax(best_logits, axis=1)
preds_threshold = np.argmax(probs, axis=1)

for c, t in thresholds.items():
    mask = probs[:, c] > t
    preds_threshold[mask] = c

f1_threshold = f1_score(val_labels, preds_threshold, average='macro')
print(f"\nF1-Macro com Thresholds: {f1_threshold:.5f}")

In [ ]:
# ===== RESUMO =====
print("\n" + "="*60)
print("📊 RESUMO - BioBERTpt Validation")
print("="*60)

all_results = [
    ('CrossEntropy', results_ce['eval_f1_macro']),
    ('Focal_g2', results_focal['eval_f1_macro']),
    ('Focal_g3', results_focal3['eval_f1_macro']),
    ('Focal_lr1e5', results_lr['eval_f1_macro']),
    ('+ Thresholds', f1_threshold),
]

for name, f1 in all_results:
    print(f"{name:<20} {f1:.5f}")

print(f"\n📝 Score anterior (CE): 0.72480")
print(f"📝 Referência (BERTimbau v4): 0.82073")